## Experiments

In [0]:
# exp = spark.read.parquet("/mnt/lsde/group05/filtered_comments/2014-3.parquet")

In [0]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

WARNING: You are using pip version 21.0.1; however, version 22.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
spark = sparknlp.start()

In [0]:
MODEL_NAME='classifierdl_use_emotion'

In [0]:
text_list = [
            """I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!""",
            """That moment when you see your friend in a commercial. Hahahaha!""",
            """My soul has just been pierced by the most evil look from @rickosborneorg. A mini panic attack &amp; chill in bones followed soon after.""",
            """For some reason I woke up thinkin it was Friday then I got to school and realized its really Monday -_-""",
            """I'd probably explode into a jillion pieces from the inablility to contain all of my if I had a Whataburger patty melt right now. #drool""",
            """These are not emotions. They are simply irrational thoughts feeding off of an emotion""",
            """Found out im gonna be with sarah bo barah in ny for one day!!! Eggcitement :)""",
            """That awkward moment when you find a perfume box full of sensors!""",
            """Just home from group celebration - dinner at Trattoria Gianni, then Hershey Felder's performance - AMAZING!!""",
            """Nooooo! My dad turned off the internet so I can't listen to band music!""",
            ]


In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
# result = pipelineModel.transform(df)
result = pipelineModel.transform(exp.withColumnRenamed("body","text"))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1512654649953687> in <module> 
 4 df = spark . createDataFrame ( pd . DataFrame ( { "text" : text_list } ) ) 
 5 # result = pipelineModel.transform(df) 
 ----> 6 result = pipelineModel . transform ( exp . withColumnRenamed ( "body" , "text" ) ) 

 NameError : name 'exp' is not defined

In [0]:
#result.show()

In [0]:
# result.select("parent_id", "author", "text", "id", "score", "ups", "subreddit", "subreddit_id", "created_utc", F.explode(F.arrays_zip('sentiment.result')).alias("cols")).show()

In [0]:
# result[1]

In [0]:
# sentiment = result.select("parent_id", "author", "text", "id", "score", "ups", "subreddit", "subreddit_id", "created_utc", F.explode(F.arrays_zip('sentiment.result')).alias("cols")).select("parent_id", "author", "text", "id", "score", "ups", "subreddit", "subreddit_id", "created_utc", F.col("cols.*"))

# sentiment.show()

## Implementation

In [0]:
def items_with_sentiment(year = 2014, month = 3, comments_or_submissions = "comments"):
    if comments_or_submissions == "comments":
        link = "filtered_comments/"
    elif comments_or_submissions == "submissions":
        link = "kwfilter_top3/"
    else:
        link = "non-exist"
    result = None
    month_literal = str(month)
    data_path = "/mnt/lsde/group05/" + link + str(year) + "-" + month_literal + ".parquet"
    result = spark.read.parquet(data_path)
    result = pipelineModel.transform(result.withColumnRenamed("body","text"))
    sentiment = result.select("parent_id", "author", "text", "id", "score", "ups", "subreddit", "subreddit_id", "created_utc", F.explode(F.arrays_zip('sentiment.result')).alias("cols")).select("parent_id", "author", "text", "id", "score", "ups", "subreddit", "subreddit_id", "created_utc", F.col("cols.*"))
    sentiment.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/comments_sentiment/" + str(year) + "-" + str(month) + ".parquet")

In [0]:
items_with_sentiment(2014, 2)

In [0]:
for year in range(2014, 2022):
    items_with_sentiment(year, 12)

In [0]:
items_with_sentiment(2022, 8)

In [0]:
# items_with_sentiment(2014, 3, "comments")

In [0]:
for month in range(3, 12):
    print("processing 2014-" + str(month))
    items_with_sentiment(2014, month)
for year in range(2015, 2022):
    for month in range(1, 13):
        print("processing " + str(year) + "-" + str(month))
        items_with_sentiment(year, month)
for month in range(1, 9):
    print("processing 2022-" + str(month))
    items_with_sentiment(2022, month)

In [0]:
def submissions_with_sentiment(year = 2014, month = 3, comments_or_submissions = "submissions"):
    if comments_or_submissions == "comments":
        link = "filtered_comments/"
    elif comments_or_submissions == "submissions":
        link = "kwfilter_top3/"
    else:
        link = "non-exist"
    result = None
    month_literal = str(month)
    data_path = "/mnt/lsde/group05/" + link + str(year) + "-" + month_literal + ".parquet"
    result = spark.read.parquet(data_path)
    result = pipelineModel.transform(result.withColumnRenamed("title","text"))
    sentiment = result.select("author", "text", "selftext", "subreddit", "id", "score", "created_utc", F.explode(F.arrays_zip('sentiment.result')).alias("cols")).select("author", "text", "selftext", "subreddit", "id", "score", "created_utc", F.col("cols.*"))
    sentiment.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/submissions_sentiment/" + str(year) + "-" + str(month) + ".parquet")

In [0]:
for month in range(2, 13):
    print("processing 2014-" + str(month))
    submissions_with_sentiment(2014, month)
for year in range(2015, 2022):
    for month in range(1, 13):
        print("processing " + str(year) + "-" + str(month))
        submissions_with_sentiment(year, month)
for month in range(1, 9):
    print("processing 2022-" + str(month))
    submissions_with_sentiment(2022, month)

processing 2014-2
processing 2014-3
processing 2014-4
processing 2014-5
processing 2014-6
processing 2014-7
processing 2014-8
processing 2014-9
processing 2014-10
processing 2014-11
processing 2014-12
processing 2015-1
processing 2015-2
processing 2015-3
processing 2015-4
processing 2015-5
processing 2015-6
processing 2015-7
processing 2015-8
processing 2015-9
processing 2015-10
processing 2015-11
processing 2015-12
processing 2016-1
processing 2016-2
processing 2016-3
processing 2016-4
processing 2016-5
processing 2016-6
processing 2016-7
processing 2016-8
processing 2016-9
processing 2016-10
processing 2016-11
processing 2016-12
processing 2017-1
processing 2017-2
processing 2017-3
processing 2017-4
processing 2017-5
processing 2017-6
processing 2017-7
processing 2017-8
processing 2017-9
processing 2017-10
processing 2017-11
processing 2017-12
processing 2018-1
processing 2018-2
processing 2018-3
processing 2018-4
processing 2018-5
processing 2018-6
processing 2018-7
processing 2018-8
processing 2018-9
processing 2018-10
processing 2018-11
processing 2018-12
processing 2019-1
processing 2019-2
processing 2019-3
processing 2019-4
processing 2019-5
processing 2019-6
processing 2019-7
processing 2019-8
processing 2019-9
processing 2019-10
processing 2019-11
processing 2019-12
processing 2020-1
processing 2020-2
processing 2020-3
processing 2020-4
processing 2020-5
processing 2020-6
processing 2020-7
processing 2020-8
processing 2020-9
processing 2020-10
processing 2020-11
processing 2020-12
processing 2021-1
processing 2021-2
processing 2021-3
processing 2021-4
processing 2021-5
processing 2021-6
processing 2021-7
processing 2021-8
processing 2021-9
processing 2021-10
processing 2021-11
processing 2021-12
processing 2022-1
processing 2022-2
processing 2022-3
processing 2022-4
processing 2022-5
processing 2022-6
processing 2022-7
processing 2022-8